## Data Preparation

In [1]:
from datasets import load_dataset
import numpy as np

import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Dataset

In [2]:
dataset = load_dataset("imagefolder", data_dir="./datasets/chest_xray")

Resolving data files: 100%|██████████| 624/624 [00:00<00:00, 311836.73it/s]


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5216
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 624
    })
})


### Setup Labels

In [4]:
labels = labels = dataset["train"].features["label"].names
print(labels)

['NORMAL', 'PNEUMONIA']


In [5]:
label2id, id2label = dict(), dict()

for i, label in enumerate(labels):
  label2id[i] = label
  id2label[label] = i

In [6]:
print(label2id)
print(id2label)

{0: 'NORMAL', 1: 'PNEUMONIA'}
{'NORMAL': 0, 'PNEUMONIA': 1}


### Transforming Data

In [7]:
from transformers import AutoImageProcessor
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

In [8]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [9]:
size = (image_processor.size["height"], image_processor.size["width"])
resizer = RandomResizedCrop(size)
normalize = Normalize(image_processor.image_mean, image_processor.image_std)

In [10]:
_transforms = Compose([resizer, ToTensor(), normalize])

In [11]:
def transforms(examples):
  examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
  del examples["image"]
  return examples

In [12]:
dataset  = dataset.with_transform(transforms)

In [13]:
print(dataset['train'])

Dataset({
    features: ['image', 'label'],
    num_rows: 5216
})


### Preparing metrics for the model

In [14]:
accuracy = evaluate.load("accuracy")

In [15]:
def compute_metrics(eval_pred):
  predictions = np.argmax(eval_pred.predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

### Setting Up Model

In [16]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
  "google/vit-base-patch16-224-in21k",
  num_labels=len(labels),
  id2label=id2label,
  label2id=label2id
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model = model.to("cuda")

In [18]:
model.device

device(type='cuda', index=0)

### Training The Model

In [19]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import DefaultDataCollator

In [20]:
training_args = TrainingArguments(
  output_dir = "pneumonia_model",
  evaluation_strategy="epoch",
  save_strategy="epoch",
  learning_rate=5e-5,
  per_device_train_batch_size=12,
  per_device_eval_batch_size=12,
  num_train_epochs=2,
  load_best_model_at_end=True,
  metric_for_best_model="accuracy",
  remove_unused_columns=False,
)

In [21]:
trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=DefaultDataCollator(),
  train_dataset=dataset["train"],
  eval_dataset=dataset["test"],
  tokenizer=image_processor,
  compute_metrics=compute_metrics
)

In [22]:
trainer.train()

                                                 
 50%|█████     | 435/870 [03:09<02:19,  3.12it/s]

{'eval_loss': 0.5674635767936707, 'eval_accuracy': 0.8333333333333334, 'eval_runtime': 11.545, 'eval_samples_per_second': 54.049, 'eval_steps_per_second': 4.504, 'epoch': 1.0}


 57%|█████▋    | 500/870 [03:37<02:33,  2.41it/s]

{'loss': 0.2272, 'learning_rate': 2.1264367816091954e-05, 'epoch': 1.15}


                                                 
100%|██████████| 870/870 [06:19<00:00,  2.94it/s]

{'eval_loss': 0.362996369600296, 'eval_accuracy': 0.8910256410256411, 'eval_runtime': 11.8579, 'eval_samples_per_second': 52.623, 'eval_steps_per_second': 4.385, 'epoch': 2.0}


100%|██████████| 870/870 [06:21<00:00,  2.28it/s]

{'train_runtime': 381.0161, 'train_samples_per_second': 27.379, 'train_steps_per_second': 2.283, 'train_loss': 0.18832303189683233, 'epoch': 2.0}


TrainOutput(global_step=870, training_loss=0.18832303189683233, metrics={'train_runtime': 381.0161, 'train_samples_per_second': 27.379, 'train_steps_per_second': 2.283, 'train_loss': 0.18832303189683233, 'epoch': 2.0})

In [23]:
dataset_val = dataset['validation'][0:10]

In [50]:
image = dataset_val["pixel_values"][0]

In [51]:
import torchvision.transforms as T

pil_transform = T.ToPILImage()
image = pil_transform(image)

image.show()

In [52]:
from transformers import pipeline

classifier = pipeline("image-classification", image_processor=image_processor, feature_extractor="google/vit-base-patch16-224-in21k", model=model)
classifier(image)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

###